<a href="https://colab.research.google.com/github/nzcbass/Pythontoopenai/blob/main/End_to_end_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports World cities and tidies it up

import pandas as pd

# Provide the path to your Excel file
xlsx_path = "worldcities.xlsx"

# Load the Excel file into a DataFrame
df1 = pd.read_excel(xlsx_path)

# Assuming your DataFrame is named df1
df1 = df1.dropna(subset=['population'])

# Remove rows where Population is less than 50,000
df1 = df1[df1['population'] >= 50000]

# Assuming your DataFrame is named df1
df1 = df1.drop(['iso2', 'iso3', 'population'], axis=1)

# Convert 'city' and 'country' columns to lowercase
df1['city'] = df1['city'].str.lower()
df1['country'] = df1['country'].str.lower()

# Remove duplicate rows based on the 'city' column
df1 = df1.drop_duplicates(subset='city', keep='first')

# Remove duplicates based on both 'city' and 'country' columns in df1
df1.drop_duplicates(subset=['city', 'country'], keep='first', inplace=True)

# Assuming you have already loaded the DataFrame
df1.loc[:, 'city'] = df1['city'].str.replace(r'\s*city$', '', case=False, regex=True)

# Copy df1 to create df2
df2 = df1.copy()

# Drop 'city' column from df2
df2 = df2.drop(['city'], axis=1)

# Remove duplicate rows based on the 'country' column from df2
df2 = df2.drop_duplicates(subset='country', keep='first')

# Display the DataFrames or perform any other operations as needed
print("df1:")
print(df1.head())

print("\ndf2:")
print(df2.head())

# Adding a new row with "US" in the "country" column
new_rows = pd.DataFrame({'country': ['us', 'usa', 'united states of america', 'uk']})
df2 = pd.concat([df2, new_rows], ignore_index=True)

# Remove duplicates in df1 based on 'city' and 'country'
df1 = df1.drop_duplicates(subset=['city', 'country'], keep='first')

# Check for duplicates in 'df1'
duplicates_df1 = df1[df1.duplicated(subset=['city', 'country'], keep=False)]

# Check for duplicates in 'df2'
duplicates_df2 = df2[df2.duplicated(subset=['country'], keep=False)]

# Remove blank rows in df1
df1 = df1.dropna()

# Remove blank rows in df2
df2 = df2.dropna()

# Display the duplicates in 'df1'
print("Duplicates in df1:")
print(duplicates_df1)

# Display the duplicates in 'df2'
print("\nDuplicates in df2:")
print(duplicates_df2)

# Count rows in df1
rows_df1 = len(df1)
print(f"Number of rows in df1: {rows_df1}")

# Count rows in df2
rows_df2 = len(df2)
print(f"Number of rows in df2: {rows_df2}")


# Save df1 to a CSV file
df1.to_csv('world_cities_countries.csv', index=False)

# Save df2 to a CSV file
df2.to_csv('world_countries.csv', index=False)


# Check if "US" is in the "country" column
is_us_in_df2 = 'us' in df2['country'].values

# print(is_us_in_df2)


In [43]:
#  Takes the json filed and creates location data etc from it

# this code is perfect, do no change
#  it loads the json, adds jon numbers
#  saves back down for later use
#  !!!!!DO NOT CHANGE!!!!!

import json
import pandas as pd
import re

# Assuming 'resume.json' is in a directory called 'resumes' in the same parent directory as your Python script
json_file_path1 = "/content/CV_parsed_by_CV_Parser_Premium-23.json"  # Relative path

# Open the JSON file and load its contents into a Python dictionary
with open(json_file_path1, 'r') as file:
    resume_data = json.load(file)

# Labeling the job entries
job_count = 1
for experience in resume_data['profile']['professional_experiences']:
    experience['job_label'] = f'Job_{job_count}'
    job_count += 1

# Save the modified data back to the JSON file
output_json_path = "/content/jobslabeled.json"
with open(output_json_path, 'w') as output_file:
    json.dump(resume_data, output_file, indent=2)

print(f"Job labeling completed. Labeled data saved to {output_json_path}")

# Now 'resume_data' contains the contents of the JSON file
# print(json.dumps(resume_data, indent=2))

# Load df1 from the Excel file
df1_path = "/content/world_cities_countries.csv"
df1 = pd.read_csv(df1_path)


def split_and_create_columns(location):
    if pd.notna(location):
        substrings = re.split(r'[^\w]+', location)
        substrings = [entry.strip() for entry in substrings]
        substrings = [remove_special_terms(entry) for entry in substrings]

        if any("justice" in entry.lower() or "precinct" in entry.lower() for entry in substrings):
            substrings.append("new zealand")

        if any("christchurch" in entry.lower() or "convention" in entry.lower() for entry in substrings):
            substrings.append("new zealand")

        return substrings

    return pd.NA

def remove_special_terms(substring):
    special_terms = ["plaza", "street", "st", "road", "rd", "avenue", "close", "motorway", "highway",
                     "po", "PO", "new zealand|po", "office", "drive", "cor.", "ave.", "ave", "level", "lvl",
                     "box", "lv", "village", "vellage", "building", "bldg", "city", "albany"]

    pattern = r'\b(?:' + '|'.join(re.escape(term) for term in special_terms) + r')\b'

    substring = re.sub(pattern, '', substring, flags=re.IGNORECASE).strip()

    if "rubber" in substring.lower():
        substring = substring.replace("rubber", "").strip()

    if "tire" in substring.lower():
        substring = substring.replace("tire", "").strip()

    return substring

if __name__ == "__main__":
    json_file_path2 = "jobslabeled.json"
    df = pd.json_normalize(json.load(open(json_file_path2, 'r'))['profile']['professional_experiences'])

    # Focus on the 'location' column
    df['location'] = df['location'].str.lower()
    new_columns = df['location'].apply(split_and_create_columns).apply(pd.Series)

    # Label the new columns with subscripts and append numbers
    new_columns.columns = [f'subscript_{i+1}' for i in range(new_columns.shape[1])]

    # Concatenate the new columns to the original DataFrame
    df = pd.concat([df, new_columns], axis=1)

# Save df to a CSV file
df.to_csv('output_data.csv', index=False)

Job labeling completed. Labeled data saved to /content/jobslabeled.json


In [44]:
import pandas as pd

# Load df & df1 & df2
df_path = "/content/output_data.csv"
df = pd.read_csv(df_path)

df1_path = "/content/world_countries.csv"
df1 = pd.read_csv(df1_path)

df2_path = "/content/world_cities_countries.csv"
df2 = pd.read_csv(df2_path)

# Convert columns to string type
df['location'] = df['location'].astype(str)
df1['country'] = df1['country'].astype(str)
df2['city'] = df2['city'].astype(str)

# Create a new column to store the found country names
# df['found_countries'] = None

# Create a new column 'countries_matched' in df
df['countries_matched'] = df['location'].apply(lambda x: ', '.join(country for country in df1['country'] if f" {country.lower()} " in f" {x.lower()} "))

# Create a new column 'cities_matched' in df
df['cities_matched'] = df['location'].apply(lambda x: ', '.join(city for city in df2['city'] if f" {city.lower()} " in f" {x.lower()} "))



# Ammendments to search criteria

# Check if 'justice' and 'precinct' are found in the 'location' column
condition = df['location'].str.contains('justice', case=False) & df['location'].str.contains('precinct', case=False)
# If the condition is met, set 'New Zealand' in the 'countries_matched' column
df.loc[condition, 'countries_matched'] = 'new zealand'




# Create a new column 'Final_match' with capitalized words and no leading spaces
df['Final_match'] = df.apply(lambda row: ' '.join([(word.upper() if len(word) == 2 else (word.capitalize() if len(word) <= 3 else word.title())) for word in row[['countries_matched', 'cities_matched']]]).lstrip(), axis=1)

# Save df as a CSV file
csv_output_path = "city_country_output_data.csv"
df.to_csv(csv_output_path, index=False)
# print(f"DataFrame saved to {csv_output_path}")

# Explicitly print matched values
for index, row in df.iterrows():
    print(f"Row {index + 1}:")
    print(f"  Countries Matched: {row['countries_matched']}")
    print(f"  Cities Matched: {row['cities_matched']}")
    print()

# Filter rows where 'city' is equal to 'ica'
filtered_rows = df2[df2['city'].apply(lambda x: f" ica " in f" {x.lower()} ")]

# Print the result
print(filtered_rows)


Row 1:
  Countries Matched: 
  Cities Matched: 

Row 2:
  Countries Matched: 
  Cities Matched: batangas

Row 3:
  Countries Matched: 
  Cities Matched: batangas

Row 4:
  Countries Matched: saudi arabia
  Cities Matched: 

Row 5:
  Countries Matched: saudi arabia
  Cities Matched: 

Row 6:
  Countries Matched: 
  Cities Matched: 

Row 7:
  Countries Matched: 
  Cities Matched: 

Row 8:
  Countries Matched: 
  Cities Matched: 

Row 9:
  Countries Matched: singapore
  Cities Matched: singapore

Row 10:
  Countries Matched: 
  Cities Matched: 

Row 11:
  Countries Matched: 
  Cities Matched: 

     city country
2307  ica    peru
